In [1]:
import requests
from pprint import pprint
import re
import lxml.html
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
def log_error(line1, line2):
    _line1 = str(line1)
    _line2 = str(line2)
    diff = 0
    chars1 = len(_line1) + 10
    chars2 = len(_line2) + 10
    if chars1 > chars2:
        diff = chars1 - chars2
        if diff % 2 != 0:
            diff = int(diff - .5)
            _line2 = '# ' + _line2
        else:
            _line2 = ' ' + _line2
        main_hashes = ''.join(['#' for i in range(chars1)])
        hashes = ''.join(['#' for i in range(int(diff/2)+4)])
        log_line = (f"{main_hashes}\n"
                    f"#### {_line1} ####\n"
                    f"{hashes}{_line2} {hashes}\n"                    
                    f"{main_hashes}\n")
        return(log_line)
   
    elif chars2 > chars1:
        diff = chars2 - chars1
        if diff % 2 != 0:
            diff = int(diff - .5)
            _line1 = '# ' + _line1
        else:
            _line1 = ' ' + _line1
        main_hashes = ''.join(['#' for i in range(chars2)])
        hashes = ''.join(['#' for i in range(int(diff/2)+4)])
        log_line = (f"{main_hashes}\n"
                    
                    f"{hashes}{_line1} {hashes}\n"
                    f"#### {_line2} ####\n"
                    f"{main_hashes}\n")
        return(log_line)
        
    else:
        hashes = '####'
        main_hashes = ''.join(['#' for i in range(chars2)])
        log_line = (f"{main_hashes}\n"
                    f"#### {_line1} ####\n"
                    f"{hashes} {_line2} {hashes}\n"
                    f"{main_hashes}\n")
        return(log_line)
        

In [3]:
# init session 
S = requests.Session()

# set URL
URL = "https://en.wikipedia.org/w/api.php"

In [22]:
page_id = '158830'
topic = 'mineral'

In [23]:
# get sections
PARAMS = {
    "action": "parse",
    "pageid": page_id,
    "format": "json",
    "prop": "sections"
}

sectionsR = S.get(url=URL, params=PARAMS)
sectionDATA = sectionsR.json()



In [24]:
sectionDATA

{'parse': {'title': 'List of minerals',
  'pageid': 158830,
  'sections': [{'toclevel': 1,
    'level': '2',
    'line': 'A',
    'number': '1',
    'index': '1',
    'fromtitle': 'List_of_minerals',
    'byteoffset': 769,
    'anchor': 'A'},
   {'toclevel': 1,
    'level': '2',
    'line': 'B',
    'number': '2',
    'index': '2',
    'fromtitle': 'List_of_minerals',
    'byteoffset': 4941,
    'anchor': 'B'},
   {'toclevel': 1,
    'level': '2',
    'line': 'C',
    'number': '3',
    'index': '3',
    'fromtitle': 'List_of_minerals',
    'byteoffset': 7429,
    'anchor': 'C'},
   {'toclevel': 1,
    'level': '2',
    'line': 'D',
    'number': '4',
    'index': '4',
    'fromtitle': 'List_of_minerals',
    'byteoffset': 11190,
    'anchor': 'D'},
   {'toclevel': 1,
    'level': '2',
    'line': 'E',
    'number': '5',
    'index': '5',
    'fromtitle': 'List_of_minerals',
    'byteoffset': 12039,
    'anchor': 'E'},
   {'toclevel': 1,
    'level': '2',
    'line': 'F',
    'number':

In [25]:
sections = [sec['anchor'] for sec in sectionDATA["parse"]["sections"]]


In [26]:
# create section list filtering out non-data sections
filter = [
    #'References',
    'See_also',
    'External_links'
]

sec_list = [sec for sec in sectionDATA["parse"]["sections"] if not sec['anchor'] in filter]


len(sec_list)

26

In [27]:
# create nested section_dict dictionary from section list based on toclevel
# adding 'Z' section by hand consider refactor
section_dict = {}
is_new_parent = True
parent = ''
for i in range(len(sec_list) - 1):
    if sec_list[i]['toclevel'] == 1 and sec_list[i + 1]['toclevel'] == 1:
        section_dict[sec_list[i]['anchor']] = []
    elif sec_list[i]['toclevel'] == 1 and sec_list[i + 1]['toclevel'] == 2:
        section_dict[sec_list[i]['anchor']] = {}
        parent = sec_list[i]['anchor']
    elif sec_list[i]['toclevel'] == 2:
        section_dict[parent][sec_list[i]['anchor']] = []
section_dict['Z'] = []
#section_dict

In [28]:
# make list of top level sections
secs = [sec for sec, val in section_dict.items()]


In [29]:
# get links
PARAMS = {
    "action": "parse",
    "pageid": page_id,
    "format": "json",
    "prop": "links"
}

linkR = S.get(url=URL, params=PARAMS)
linkDATA = linkR.json()

pprint(linkDATA["parse"]["links"])

[{'*': 'Portal:Geology', 'exists': '', 'ns': 100},
 {'*': 'Portal:Earth sciences', 'exists': '', 'ns': 100},
 {'*': 'List of minerals (complete)', 'exists': '', 'ns': 0},
 {'*': 'Abelsonite', 'exists': '', 'ns': 0},
 {'*': 'Abenakiite-(Ce)', 'exists': '', 'ns': 0},
 {'*': 'Abernathyite', 'exists': '', 'ns': 0},
 {'*': 'Abhurite', 'exists': '', 'ns': 0},
 {'*': 'Abramovite', 'exists': '', 'ns': 0},
 {'*': 'Abswurmbachite', 'exists': '', 'ns': 0},
 {'*': 'Acanthite', 'exists': '', 'ns': 0},
 {'*': 'Achavalite', 'exists': '', 'ns': 0},
 {'*': 'Actinolite', 'exists': '', 'ns': 0},
 {'*': 'Acuminite', 'exists': '', 'ns': 0},
 {'*': 'Adamantine spar', 'exists': '', 'ns': 0},
 {'*': 'Adamite', 'exists': '', 'ns': 0},
 {'*': 'Adamsite-(Y)', 'exists': '', 'ns': 0},
 {'*': 'Adelite', 'exists': '', 'ns': 0},
 {'*': 'Admontite', 'exists': '', 'ns': 0},
 {'*': 'Aegirine', 'exists': '', 'ns': 0},
 {'*': 'Aenigmatite', 'exists': '', 'ns': 0},
 {'*': 'Aerinite', 'exists': '', 'ns': 0},
 {'*': 'Aerugit

In [30]:
links = [link['*'] for link in linkDATA["parse"]["links"]]


In [31]:
for link in links[3:4]:
    # get page info
    PARAMS = {
        "action": "query",
        "titles": link,
        "format": "json",
        "prop": "info",
        'inprop': "displaytitle"
    }

    infoR = S.get(url=URL, params=PARAMS)
    infoDATA = infoR.json()
    pprint(infoDATA)

{'batchcomplete': '',
 'query': {'pages': {'2478814': {'contentmodel': 'wikitext',
                                 'displaytitle': 'Abelsonite',
                                 'lastrevid': 918628508,
                                 'length': 7818,
                                 'ns': 0,
                                 'pageid': 2478814,
                                 'pagelanguage': 'en',
                                 'pagelanguagedir': 'ltr',
                                 'pagelanguagehtmlcode': 'en',
                                 'title': 'Abelsonite',
                                 'touched': '2019-12-23T21:57:44Z'}}}}


In [32]:
# get data
PARAMS = {
    "action": "parse",
    "pageid": page_id,
    "format": "json",
    "prop": "wikitext"
}

textR = S.get(url=URL, params=PARAMS)
textDATA = textR.json()



In [33]:
# parse data

lines = textDATA["parse"]["wikitext"]["*"].split('\n')

In [34]:
items = []

with open(topic + '_error_log.txt', 'w+', encoding="utf-8") as filepath:
    for index, line in enumerate(lines[20:1708]):
        
        if re.match('\*\[\[', line[0:3]):
            try:
                item = re.search(r'\[\[(.*?)\]\]', line).group(1)
                items.append(item)
            except:
                topic_error_line = f"{topic} parse error"
                item_line = f"{str(index)}, {line}"
                print(log_error(topic_error_line, item_line))
            


In [37]:
item_infos = []
for index, title in enumerate(items):
    with open('info_error_log.txt', 'w+', encoding="utf-8") as filepath:
        
        url = 'https://en.wikipedia.org/w/index.php?title=' + title + '&action=info'

        #print(url)
        response = requests.get(url)
        doc = lxml.html.fromstring(response.content)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        try:
            # info_table = doc.xpath('//*[@class="wikitable mw-page-info"]')[0]
            watchers = doc.xpath('//*[@id="mw-pageinfo-watchers"]')[0][1].text
            
        except:
            topic_error_line = f"watchers parse error"
            item_line = f"{str(index)}, {title}, {url}"
            print(log_error(topic_error_line, item_line))
            filepath.write(str(index) + ' : ' + title + ' : ' + url + '\n')
            watchers = "#error#"

        try:
            redirects = doc.xpath('//*[contains(text(),"redirects")]/../../td')[1].text
            #redirects = doc.xpath('//*[contains(text(),"redirects")]/parent::node()/parent::node()')
            #redirects = doc.xpath('/html/body/div[3]/div[3]/div[3]/table[2]/tr[9]/td')[1].text
            
        except:
            topic_error_line = f"redirects parse error"
            item_line = f"{str(index)}, {title}, {url}"
            print(log_error(topic_error_line, item_line))
            filepath.write(str(index) + ' : ' + title + ' : ' + url + '\n')
            redirects = "#error#"
             
        try:
            views = doc.xpath('//*[@id="mw-pvi-month-count"]/td/div')[0].text
            
        except:
            topic_error_line = f"views parse error"
            item_line = f"{str(index)}, {title}, {url}"
            print(log_error(topic_error_line, item_line))
            filepath.write(str(index) + ' : ' + title + ' : ' + url + '\n')
            views = "#error#"        
        
        try:
            edits = doc.xpath('//*[@id="mw-pageinfo-edits"]/td')[1].text
            #edits = doc.xpath('/html/body/div[3]/div[3]/div[3]/table[4]/tr[5]/td')[1].text
    
        except:
            topic_error_line = f"edits parse error"
            item_line = f"{str(index)}, {title}, {url}"
            print(log_error(topic_error_line, item_line))
            filepath.write(str(index) + ' : ' + title + ' : ' + url + '\n')
            edits = "#error#"
        
        item_infos.append({
            topic: title,
            'watchers': watchers,
            'redirects': redirects,
            'views': views,
            'edits': edits
        })
    

KeyboardInterrupt: 

In [18]:
# transfer item_infos to json file
import json
with open(f"../output/{topic} + _infos.json", 'w') as filepath:
    json.dump(item_infos, filepath)

In [19]:
error_items = [item['mineral'] for item in item_infos if item['views'] == '#error#']

for index, title in enumerate(error_items):
    with open('info_error_log.txt', 'w+', encoding="utf-8") as filepath:
        #print(title)
        del_idx = 0
        for index, item in enumerate(item_infos):
            if item[topic] == title:
                del_idx = index
                #del item_infos[index]
        #print(item)
        
        
        if re.match(r'\W*', title):
            print('%%%')
            search = re.search(r'(\w+)?', title).group(1)
            print(search)
            title = search
        
        print(title)
        
        url = 'https://en.wikipedia.org/w/index.php?title=' + title + '&action=info'

        #print(url)
        response = requests.get(url)
        doc = lxml.html.fromstring(response.content)
        soup = BeautifulSoup(response.content, 'html.parser')
  
        try:
            # info_table = doc.xpath('//*[@class="wikitable mw-page-info"]')[0]
            watchers = doc.xpath('//*[@id="mw-pageinfo-watchers"]')[0][1].text
            
        except:
            topic_error_line = f"watchers parse error"
            item_line = f"{str(index)}, {title}, {url}"
            print(log_error(topic_error_line, item_line))
            filepath.write(str(index) + ' : ' + title + ' : ' + url + '\n')
            watchers = "#error#"

        try:
            redirects = doc.xpath('//*[contains(text(),"redirects")]/../../td')[1].text
            #redirects = doc.xpath('//*[contains(text(),"redirects")]/parent::node()/parent::node()')
            #redirects = doc.xpath('/html/body/div[3]/div[3]/div[3]/table[2]/tr[9]/td')[1].text
            
        except:
            topic_error_line = f"redirects parse error"
            item_line = f"{str(index)}, {title}, {url}"
            print(log_error(topic_error_line, item_line))
            filepath.write(str(index) + ' : ' + title + ' : ' + url + '\n')
            redirects = "#error#"
             
        try:
            views = doc.xpath('//*[@id="mw-pvi-month-count"]/td/div')[0].text
            
        except:
            topic_error_line = f"views parse error"
            item_line = f"{str(index)}, {title}, {url}"
            print(log_error(topic_error_line, item_line))
            filepath.write(str(index) + ' : ' + title + ' : ' + url + '\n')
            views = "#error#"        
        
        try:
            edits = doc.xpath('//*[@id="mw-pageinfo-edits"]/td')[1].text
            #edits = doc.xpath('/html/body/div[3]/div[3]/div[3]/table[4]/tr[5]/td')[1].text
    
        except:
            topic_error_line = f"edits parse error"
            item_line = f"{str(index)}, {title}, {url}"
            print(log_error(topic_error_line, item_line))
            filepath.write(str(index) + ' : ' + title + ' : ' + url + '\n')
            edits = "#error#"
            
        item_infos[del_idx] = {
            topic: title,
            'watchers': watchers,
            'redirects': redirects,
            'views': views,
            'edits': edits
        }

In [39]:
with open(f"../output/{topic}_infos.json", 'r') as jfile:
    data=jfile.read()

# parse file
item_infos = json.loads(data)

In [40]:
_item_infos = item_infos
len(_item_infos)

1515

In [41]:
#item_infos = [{item['views']: int(item['views'].replace(',', ''))} for item in item_infos]
for index, item in enumerate(_item_infos):
    if item['views'] != '#error#':
        if item['watchers'] == 'Fewer than 30 watchers':
            item['watchers'] = '29'
        item['views'] = int(item['views'].replace(',', ''))
        item['watchers'] = int(item['watchers'].replace(',', ''))
        item['edits'] = int(item['edits'].replace(',', ''))
        
        
        

In [58]:
by_views = sorted(_item_infos, key=lambda k : k['views'], reverse=True)
for index, item in enumerate(by_views):
    item['view_rank'] = index

In [59]:
by_edits = sorted(_item_infos, key=lambda k : k['edits'], reverse=True)
for index, item in enumerate(by_edits):
    item['edit_rank'] = index

In [60]:
by_watchers = sorted(_item_infos, key=lambda k : k['watchers'], reverse=True)
for index, item in enumerate(by_watchers):
    item['watcher_rank'] = index

In [61]:
for item in _item_infos:
    item['total_rank'] = (item['view_rank'] + item['edit_rank'] + item['watcher_rank']) / 3
    #item['total_rank'] = (item['view_rank'] + item['watcher_rank']) / 2

    

In [63]:
min_df = pd.DataFrame(_item_infos)
min_df.head()
min_df.sort_values(by=['total_rank'], ascending=False).head(100)

,edit_rank,edits,mineral,redirects,total_rank,view_rank,views,watcher_rank,watchers
71,0,7828,Aluminium,31,1.000000,1,111326,2,555
579,2,6693,Gold,32,1.000000,0,132301,1,861
437,5,5644,Diamond,24,2.666667,3,89458,0,954
1221,3,6323,Silver,22,4.000000,6,73138,3,465
1275,4,6175,Sulfur,39,4.333333,5,80155,4,435
797,1,7442,Lead,18,6.000000,9,70053,8,384
1343,6,4308,Titanium,12,6.666667,8,72050,6,400
165,7,4187,Asbestos,19,7.000000,2,100263,12,320
1219,10,4013,Silicon,21,8.000000,7,72524,7,397
1050,8,4166,Platinum,14,10.666667,15,56664,9,378


In [69]:
min_df = pd.DataFrame(_item_infos)
min_df.head()
min_df.sort_values(by=['views'], ascending=False).head(100)

,edit_rank,edits,mineral,redirects,total_rank,view_rank,views,watcher_rank,watchers
579,2,6693,Gold,32,1.000000,0,132301,1,861
71,0,7828,Aluminium,31,1.000000,1,111326,2,555
165,7,4187,Asbestos,19,7.000000,2,100263,12,320
437,5,5644,Diamond,24,2.666667,3,89458,0,954
1367,15,3085,Tungsten,17,10.666667,4,83659,13,317
1275,4,6175,Sulfur,39,4.333333,5,80155,4,435
1221,3,6323,Silver,22,4.000000,6,73138,3,465
1219,10,4013,Silicon,21,8.000000,7,72524,7,397
1343,6,4308,Titanium,12,6.666667,8,72050,6,400
797,1,7442,Lead,18,6.000000,9,70053,8,384


In [68]:
min_df.sort_values(by=['watchers'], ascending=False).head(100)

,edit_rank,edits,mineral,redirects,total_rank,view_rank,views,watcher_rank,watchers
437,5,5644,Diamond,24,2.666667,3,89458,0,954
579,2,6693,Gold,32,1.000000,0,132301,1,861
71,0,7828,Aluminium,31,1.000000,1,111326,2,555
1221,3,6323,Silver,22,4.000000,6,73138,3,465
1275,4,6175,Sulfur,39,4.333333,5,80155,4,435
1100,17,2906,Quartz,30,11.000000,11,58632,5,422
1343,6,4308,Titanium,12,6.666667,8,72050,6,400
1219,10,4013,Silicon,21,8.000000,7,72524,7,397
797,1,7442,Lead,18,6.000000,9,70053,8,384
1050,8,4166,Platinum,14,10.666667,15,56664,9,378


In [67]:
min_df.sort_values(by=['edits'], ascending=False).head(100)

,edit_rank,edits,mineral,redirects,total_rank,view_rank,views,watcher_rank,watchers
71,0,7828,Aluminium,31,1.000000,1,111326,2,555
797,1,7442,Lead,18,6.000000,9,70053,8,384
579,2,6693,Gold,32,1.000000,0,132301,1,861
1221,3,6323,Silver,22,4.000000,6,73138,3,465
1275,4,6175,Sulfur,39,4.333333,5,80155,4,435
437,5,5644,Diamond,24,2.666667,3,89458,0,954
1343,6,4308,Titanium,12,6.666667,8,72050,6,400
165,7,4187,Asbestos,19,7.000000,2,100263,12,320
1050,8,4166,Platinum,14,10.666667,15,56664,9,378
121,9,4083,Arsenic,10,12.333333,14,56784,14,304


In [53]:
by_total = sorted(_item_infos, key=lambda k : k['total_rank'], reverse=False)

In [54]:
dedupe = []
for item in by_total:
    if item[topic] not in dedupe:
        dedupe.append(item[topic])
        

In [55]:
start = 0
limit = 44
top_items = dedupe[start:limit]

In [56]:
#limit = 'all'
with open(f"../output/{topic}_{limit}.txt", 'w+', encoding="utf-8") as fp:
    for item in top_items:
        fp.write(item + '\n')